In [ ]:
import os
from dotenv import load_dotenv
from eodag import EODataAccessGateway

# Load environment variables from .env file
load_dotenv(dotenv_path="../.env")

# --- CREDENTIALS ---
username = os.getenv("EODAG__COPERNICUS_DATASPACE__AUTH__CREDENTIALS__USERNAME")
password = os.getenv("EODAG__COPERNICUS_DATASPACE__AUTH__CREDENTIALS__PASSWORD")

print("1. Initializing NETRA (Standard Mode)...")
# We initialize normally to load the "copernicus_dataspace" structure
dag = EODataAccessGateway()

# --- THE SURGERY ---
print("2. Forcing credentials into memory...")
try:
    # We access the internal dictionary directly. No parsing. No YAML.
    # We locate the specific provider and overwrite its password slot.
    provider = dag.providers_config["copernicus_dataspace"]
    
    provider.auth.credentials["username"] = username
    provider.auth.credentials["password"] = password
    provider.priority = 100  # Make it the absolute king
    
    print("✅ Surgery Successful: Credentials injected directly.")
    
except KeyError:
    print("❌ Critical Error: NETRA does not have a 'copernicus_dataspace' slot.")
except Exception as e:
    print(f"❌ Critical Error: {e}")


# --- THE TEST ---
print("\n3. Testing Vision...")
# We search specifically using the provider we just hacked
results = dag.search(
    productType='S2_MSI_L1C',
    start='2024-01-01', 
    end='2024-01-03',
    geom={'lonmin': 85.8, 'latmin': 20.2, 'lonmax': 85.9, 'latmax': 20.3},
    provider='copernicus_dataspace' # <--- WE DEMAND THIS SPECIFIC PROVIDER
)

if len(results) > 0:
    img = results[0]
    print(f"🎉 SUCCESS! Provider: {img.provider}")
    if 'thumbnail' in img.assets:
        print(f"🖼️ Thumbnail: {img.assets['thumbnail']['href']}")
    else:
        print("⚠️ No thumbnail found (Authentication might still be rejected by the server).")
else:
    print("❌ No results found. The server might have rejected the password.")